In [1]:
import argparse
import json
from torch.utils.data import DataLoader
from thop import profile
from ptflops import get_model_complexity_info

In [2]:
from models import *
from utils.datasets import *
from utils.utils import *
from quant_dorefa import *

from mymodel import *
from torchsummary import summary

In [3]:
model = torch.load('weights/4w4a/test_best.pt', map_location=torch.device('cpu'))
# model = UltraNet()
# model.load_state_dict(checkpoint[])
# model.hyp = hyp
# model.nc = 1
# model.arc = 'default'

In [ ]:
# print(model.parameters())
# print(model['model'])
# for layers in model:
#     print(layers)
# summary(model, (3, 320, 160))

In [24]:
print("Model's state_dict:")
for param_tensor in model['model']:
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
#     if(len(model['model'][param_tensor].size()) > 1):
    print(param_tensor, model['model'][param_tensor].size())

# print(model['model']['layers.0.weight'])    

Model's state_dict:
layers.0.weight torch.Size([16, 3, 3, 3])
layers.1.weight torch.Size([16])
layers.1.bias torch.Size([16])
layers.1.running_mean torch.Size([16])
layers.1.running_var torch.Size([16])
layers.1.num_batches_tracked torch.Size([])
layers.4.weight torch.Size([32, 16, 3, 3])
layers.5.weight torch.Size([32])
layers.5.bias torch.Size([32])
layers.5.running_mean torch.Size([32])
layers.5.running_var torch.Size([32])
layers.5.num_batches_tracked torch.Size([])
layers.8.weight torch.Size([64, 32, 3, 3])
layers.9.weight torch.Size([64])
layers.9.bias torch.Size([64])
layers.9.running_mean torch.Size([64])
layers.9.running_var torch.Size([64])
layers.9.num_batches_tracked torch.Size([])
layers.12.weight torch.Size([64, 64, 3, 3])
layers.13.weight torch.Size([64])
layers.13.bias torch.Size([64])
layers.13.running_mean torch.Size([64])
layers.13.running_var torch.Size([64])
layers.13.num_batches_tracked torch.Size([])
layers.16.weight torch.Size([64, 64, 3, 3])
layers.17.weight to

In [5]:
conv_0_weight = model['model']['layers.0.weight']
conv_1_weight = model['model']['layers.4.weight']
conv_2_weight = model['model']['layers.8.weight']
conv_3_weight = model['model']['layers.12.weight']
conv_4_weight = model['model']['layers.16.weight']
conv_5_weight = model['model']['layers.19.weight']
conv_6_weight = model['model']['layers.22.weight']
conv_7_weight = model['model']['layers.25.weight']
conv_8_weight = model['model']['layers.28.weight']
print(conv_0_weight)
# torch.save(conv_0_weight, 'conv_0_original.txt')

tensor([[[[ 2.14157e-01,  5.06902e-01, -1.08446e+00],
          [-2.02708e+00, -2.30518e-01,  9.73781e-02],
          [ 7.53078e-01,  7.48555e-01, -8.04692e-01]],

         [[ 7.32584e-02, -3.40585e-01,  4.14190e-01],
          [-2.20138e+00, -1.36242e+00,  2.28518e-01],
          [ 1.22073e-01,  2.14735e-01,  6.18694e-01]],

         [[ 5.39542e-01,  3.19474e-01,  7.78630e-01],
          [-2.16503e+00, -1.49110e+00,  7.00551e-02],
          [-7.31038e-01, -4.90036e-01,  6.51057e-01]]],


        [[[ 7.48441e-01, -5.18872e-01, -6.18930e-02],
          [-7.49757e-01, -1.03272e+00, -1.05459e+00],
          [-1.01737e+00, -1.18523e+00, -1.37476e+00]],

         [[-3.65692e-01,  1.05881e-04, -9.60202e-01],
          [-9.75848e-01,  3.84589e-01, -2.14345e-01],
          [-7.07630e-02, -6.60180e-01, -6.85626e-01]],

         [[-4.24429e-01,  1.55890e+00,  1.76132e+00],
          [ 1.34439e+00,  2.12045e+00,  2.43022e+00],
          [ 1.69067e+00,  2.04686e+00,  2.20378e+00]]],


        [[[ 

In [30]:
bn_1_weight = model['model']['layers.1.weight']
bn_5_weight = model['model']['layers.5.weight']
bn_9_weight = model['model']['layers.9.weight']
bn_13_weight = model['model']['layers.13.weight']
bn_17_weight = model['model']['layers.17.weight']
bn_20_weight = model['model']['layers.20.weight']
bn_23_weight = model['model']['layers.23.weight']
bn_26_weight = model['model']['layers.26.weight']

bn_1_bias = model['model']['layers.1.bias']
bn_5_bias = model['model']['layers.5.bias']                           
bn_9_bias = model['model']['layers.9.bias']                          
bn_13_bias = model['model']['layers.13.bias']                                       
bn_17_bias = model['model']['layers.17.bias']                          
bn_20_bias = model['model']['layers.20.bias']                          
bn_23_bias = model['model']['layers.23.bias']                          
bn_26_bias = model['model']['layers.26.bias']                          
print(bn_1_weight)
print(bn_1_bias)
print(bn_26_bias)

tensor([0.58797, 0.44178, 0.43090, 0.47215, 0.38584, 0.72237, 0.29577, 0.29867, 0.29386, 0.24101, 0.74779, 0.34900, 0.73918, 0.27813, 0.66866, 0.31062])
tensor([-0.69709,  0.26714,  4.11842,  0.14098,  0.03640, -0.12954, -0.01322, -0.06990, -0.00508, -0.04382, -1.08639,  0.20204, -3.47873, -0.03349,  4.08009,  0.10539])
tensor([ 0.09793,  0.11844, -0.42521,  0.19618,  0.59334, -0.70063,  0.18355,  0.21076,  0.10918,  0.80603, -0.00823,  0.09663,  0.13834,  1.53147, -0.37850,  2.07403,  0.21894,  0.11065,  0.16029,  0.12685, -1.00109,  0.15277,  0.57252,  0.15685,  0.46208,  0.03184,  0.23149,  0.10892,  0.25701,  0.16934,  0.24810,
         0.09251,  1.28291,  0.14899,  0.12189,  1.02823,  0.36970,  0.55793,  0.22179,  0.30234,  0.05277,  1.35015,  0.10731,  2.04937, -0.05521,  0.11958,  0.26256,  0.60039,  0.28685,  0.13947,  1.21211,  1.08796,  0.21183,  0.10261,  0.36136,  0.17212,  0.08371,  0.19401,  1.20058,  0.09407,  0.12747,  0.57194,
         0.14694,  0.19709])


In [26]:
def get_w_quant(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    out = torch.round(in_f * n) / n
    return out

def get_w_int(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    out = torch.round(in_f * n)
    return out

In [35]:
bn_1_weight_int = get_w_int(bn_1_weight, 14)
bn_5_weight_int = get_w_int(bn_1_weight, 13)
bn_9_weight_int = get_w_int(bn_1_weight, 12)
bn_13_weight_int = get_w_int(bn_1_weight, 11)
bn_17_weight_int = get_w_int(bn_1_weight, 13)
bn_20_weight_int = get_w_int(bn_1_weight, 12)
bn_23_weight_int = get_w_int(bn_1_weight, 13)
bn_26_weight_int = get_w_int(bn_1_weight, 14)

bn_1_bias_int = get_w_int(bn_1_bias,26)
bn_5_bias_int = get_w_int(bn_1_bias,21)
bn_9_bias_int = get_w_int(bn_1_bias,22)
bn_13_bias_int = get_w_int(bn_1_bias,21)
bn_17_bias_int = get_w_int(bn_1_bias,21)
bn_20_bias_int = get_w_int(bn_1_bias,21)
bn_23_bias_int = get_w_int(bn_1_bias,22)
bn_26_bias_int = get_w_int(bn_1_bias,23)

np.save('bn_0_weight', bn_1_weight_int)
np.save('bn_0_bias', bn_1_bias_int)
np.save('bn_1_weight', bn_5_weight_int)
np.save('bn_1_bias', bn_5_bias_int)
np.save('bn_2_weight', bn_9_weight_int)
np.save('bn_2_bias', bn_9_bias_int)
np.save('bn_3_weight', bn_13_weight_int)
np.save('bn_3_bias', bn_13_bias_int)
np.save('bn_4_weight', bn_17_weight_int)
np.save('bn_4_bias', bn_17_bias_int)
np.save('bn_5_weight', bn_20_weight_int)
np.save('bn_5_bias', bn_20_bias_int)
np.save('bn_6_weight', bn_23_weight_int)
np.save('bn_6_bias', bn_23_bias_int)
np.save('bn_7_weight', bn_26_weight_int)
np.save('bn_7_bias', bn_26_bias_int)

In [37]:
bn_3_b = np.load('bn_3_weight.npy')
print(bn_3_b)

[        541         425         415         450         376         633         294         297         292         242         648         343         643         277         598         308]


In [ ]:
def print_hex_line(in_f, size_value, SIMD, PE):
    print(size_value)
    print('{', end='')
    for i in range(0, size_value[0]):  # OUT_channel traverse
        if i%(size_value[0]/PE) == 0:
            print('')
            print('{', end = '')
        for k1 in range(0, size_value[2]): # kernel trverse
            for k2 in range(0, size_value[3]): # kernel traverse
                for j in range(0, size_value[1]): # In_channel traverse
                    if in_f[i][j][k1][k2] < 0:
                        in_f[i][j][k1][k2] = in_f[i][j][k1][k2] * (-1) + 8
                    if j/(size_value[1]/SIMD) == 0:
                        print('"0x', end='')
                    formatted_data = format(int(in_f[i][j][k1][k2].item()), "x")
#                     print(formatted_data, end = '')
                    print(i, end='')
                    if size_value[1] <= SIMD:
                        if i == size_value[0]/PE -1:
                            print('"}', end='')
                        else:
                            if j == size_value[1]-1:
                                print('",', end='')
#                             else:
                    else:
                        if int((j+1)%(size_value[1]/SIMD)) == 0:
                            print('",', end='')
    print('}', end = '')

In [ ]:
def weight_reorg_2d(in_f, size_value, SIMD, PE):
    print(size_value)
    K = size_value[2]
    IN_CH = size_value[1]
    OUT_CH = size_value[0]
#     weight_store[OUT_CH][OUT_CH*IN_CH*K*K]
    rows, cols = (OUT_CH, IN_CH*K*K)
    weight_store = [[0 for i in range(cols)] for j in range(rows)]
#     dim_1 = PE
#     dim_2 = K*K*OUT_CH/PE*IN_CH*SIMD
    
    for out_channel in range(0, OUT_CH):
        for in_channel in range(0, IN_CH):
            for ker1 in range(0, K):
                for ker2 in range(0, K):
                    weight_store[out_channel][in_channel*K*K + ker1*K + ker2] = int(in_f[out_channel][in_channel][ker1][ker2].item());
    return weight_store, rows, cols

def weight_print(weight_2d, rows, cols, SIMD, PE):
    print('out_dim = ', int(PE), ',', int((cols/SIMD)*(rows/PE)))
    print('{')
    for i in range(0, rows):
        if i%(rows/PE) == 0:
            print('')
            print('{')
        simd_cnt = 0
        for j in range(0, cols):
            if weight_2d[i][j] < 0:
                weight_2d[i][j] = weight_2d[i][j] * (-1) + 8
            if j%SIMD == 0:
                print('"0x', end='')
            formatted_data = format(int(weight_2d[i][j]), "x")
            print(formatted_data, end='')
            simd_cnt = simd_cnt + 1
            if simd_cnt%SIMD == 0:
                if j == cols-1 and (i+1)%(rows/PE)==0:
                    if j == cols -1 and i == rows-1:
                        print('"}')
                    else:
                        print('"},', end='')
                else:
                    print('",', end='')
    print('}')

In [22]:
conv_0_wq = get_w_int(conv_8_weight, 4)
np.save('conv_8',conv_0_wq)
# print(conv_0_wq)

In [23]:
a = np.load('conv_8.npy')
print(a)

[[[[         -1]]

  [[          1]]

  [[         -0]]

  ...

  [[         -0]]

  [[          1]]

  [[          0]]]


 [[[         -2]]

  [[          6]]

  [[         -0]]

  ...

  [[          6]]

  [[         -0]]

  [[          6]]]


 [[[          3]]

  [[          1]]

  [[         -0]]

  ...

  [[         -0]]

  [[          3]]

  [[         -0]]]


 ...


 [[[         -2]]

  [[          1]]

  [[          0]]

  ...

  [[          0]]

  [[         -1]]

  [[         -5]]]


 [[[          4]]

  [[         -5]]

  [[         -5]]

  ...

  [[         -6]]

  [[         -4]]

  [[         -2]]]


 [[[          0]]

  [[          0]]

  [[         -0]]

  ...

  [[          0]]

  [[         -1]]

  [[          0]]]]


In [ ]:
conv_0_wq = get_w_int(conv_8_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
# conv_0_wq = conv_0_wq.cpu()
# conv_0_wq = get_w_int(conv_0_wq, 4)
weight_store, rows, cols = weight_reorg_2d(conv_0_wq, size_value, 0, 0)
print(rows, cols)
weight_print(weight_store, rows, cols, SIMD, PE)

In [ ]:
conv_0_wq = get_w_int(conv_0_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
conv_0_wq = conv_0_wq.cpu()
print_hex_line(conv_0_wq.data, size_value, SIMD, PE)
# print(size_value[1])

In [ ]:
print(size_value)